# Image as input

In [6]:
import os
import torch
from PIL import Image, ImageDraw, ImageFont
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as F

# ===== HARD-CODED PARAMETERS =====
IMAGE_FOLDER = r"archive/dataset/dataset/test/images"
OUTPUT_FOLDER = r"results"
CHECKPOINT_PATH = r"ckpt/faster_rcnn_epoch_1.pth"
NUM_CLASSES = 4  # 包含 background
LABELS = ["background", "pothole", "cracks", "open_manhole"]
THRESHOLD = 0.5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# =================================

# 支援的影像副檔名
IMG_EXTENSIONS = {'.jpg', '.jpeg', '.png', '.bmp'}

def get_model(num_classes, checkpoint_path=None, device='cpu'):
    model = fasterrcnn_resnet50_fpn(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    if checkpoint_path and os.path.exists(checkpoint_path):
        state_dict = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    return model


def predict(model, image, device, threshold=0.5):
    img_tensor = F.to_tensor(image).to(device)
    with torch.no_grad():
        outputs = model([img_tensor])[0]

    keep = outputs['scores'] >= threshold
    boxes = outputs['boxes'][keep].cpu()
    labels = outputs['labels'][keep].cpu()
    scores = outputs['scores'][keep].cpu()
    return boxes, labels, scores


def draw_boxes(image, boxes, labels, scores, category_names=None):
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    for box, label, score in zip(boxes, labels, scores):
        x1, y1, x2, y2 = box.tolist()
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
        text = f"{category_names[label] if category_names else label}: {score:.2f}"
        text_bbox = draw.textbbox((x1, y1), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        draw.rectangle([x1, y1 - text_height, x1 + text_width, y1], fill="red")
        draw.text((x1, y1 - text_height), text, fill="white", font=font)
    return image


def main():
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    model = get_model(NUM_CLASSES, CHECKPOINT_PATH, DEVICE)

    for fname in os.listdir(IMAGE_FOLDER):
        ext = os.path.splitext(fname)[1].lower()
        if ext not in IMG_EXTENSIONS:
            continue
        image_path = os.path.join(IMAGE_FOLDER, fname)
        image = Image.open(image_path).convert("RGB")

        boxes, labels, scores = predict(model, image, DEVICE, THRESHOLD)
        result = draw_boxes(image.copy(), boxes, labels, scores, LABELS)

        out_path = os.path.join(OUTPUT_FOLDER, fname)
        result.save(out_path)
        print(f"Processed {fname} -> {out_path}")

if __name__ == "__main__":
    main()

c:\Users\user\anaconda3\envs\torch312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\anaconda3\envs\torch312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\user\AppData\Local\Temp\ipykernel_3772\1622032603.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `wei

Processed 1000_F_256252609_6WIHRGbpzSaVQwioubxwgXdSJTNONNcK.jpg -> results\1000_F_256252609_6WIHRGbpzSaVQwioubxwgXdSJTNONNcK.jpg
Processed 360_F_916445076_KAGVDE2nuHOIxNARmepMJTgHQqd9Y7y0.jpg -> results\360_F_916445076_KAGVDE2nuHOIxNARmepMJTgHQqd9Y7y0.jpg
Processed closemahole1.jpg -> results\closemahole1.jpg
Processed cracking-(1).jpg -> results\cracking-(1).jpg
Processed cracking-1-(1).jpg -> results\cracking-1-(1).jpg
Processed cracks3.jpeg -> results\cracks3.jpeg
Processed depositphotos_53437389-stock-photo-road.jpg -> results\depositphotos_53437389-stock-photo-road.jpg
Processed DSC00004.jpg -> results\DSC00004.jpg
Processed file-16089009.jpg -> results\file-16089009.jpg
Processed images (1).jpeg -> results\images (1).jpeg
Processed images (2).jpeg -> results\images (2).jpeg
Processed images (4).jpeg -> results\images (4).jpeg
Processed images (5).jpeg -> results\images (5).jpeg
Processed images.jpeg -> results\images.jpeg
Processed images.jpg -> results\images.jpg
Processed istoc

# Video as input

In [ ]:
import os
import cv2
import numpy as np
import torch
from PIL import Image, ImageDraw, ImageFont
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as F

# ===== HARD-CODED PARAMETERS =====
VIDEO_FOLDER = r"archive/dataset/dataset/test/video"
OUTPUT_FOLDER = r"video_results"
CHECKPOINT_PATH = r"ckpt/faster_rcnn_epoch_1.pth"
NUM_CLASSES = 4  # 包含 background
LABELS = ["background", "pothole", "cracks", "open_manhole"]
THRESHOLD = 0.5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# ================================

# 支援的影片副檔名
VID_EXTENSIONS = {'.mp4', '.avi', '.mov', '.mkv'}


def get_model(num_classes, checkpoint_path=None, device='cpu'):
    model = fasterrcnn_resnet50_fpn(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    if checkpoint_path and os.path.exists(checkpoint_path):
        state_dict = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    return model


def predict(model, image, device, threshold=0.5):
    img_tensor = F.to_tensor(image).to(device)
    with torch.no_grad():
        outputs = model([img_tensor])[0]
    keep = outputs['scores'] >= threshold
    boxes = outputs['boxes'][keep].cpu()
    labels = outputs['labels'][keep].cpu()
    scores = outputs['scores'][keep].cpu()
    return boxes, labels, scores


def draw_boxes_pil(image, boxes, labels, scores, category_names=None):
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    for box, label, score in zip(boxes, labels, scores):
        x1, y1, x2, y2 = box.tolist()
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
        text = f"{category_names[label] if category_names else label}: {score:.2f}"
        text_bbox = draw.textbbox((x1, y1), text, font=font)
        tw = text_bbox[2] - text_bbox[0]
        th = text_bbox[3] - text_bbox[1]
        draw.rectangle([x1, y1 - th, x1 + tw, y1], fill="red")
        draw.text((x1, y1 - th), text, fill="white", font=font)
    return image


def process_video(model, video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(rgb)
        boxes, labels, scores = predict(model, pil_img, DEVICE, THRESHOLD)
        inf_img = draw_boxes_pil(pil_img.copy(), boxes, labels, scores, LABELS)
        inf_frame = cv2.cvtColor(np.array(inf_img), cv2.COLOR_RGB2BGR)
        out.write(inf_frame)
        frame_idx += 1
        if frame_idx % 50 == 0:
            print(f"[{os.path.basename(video_path)}] Processed {frame_idx} frames...")

    cap.release()
    out.release()
    print(f"Finished {os.path.basename(video_path)}, saved to {output_path}")


def main():
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    model = get_model(NUM_CLASSES, CHECKPOINT_PATH, DEVICE)

    video_files = [f for f in os.listdir(VIDEO_FOLDER)
                   if os.path.splitext(f)[1].lower() in VID_EXTENSIONS]
    if not video_files:
        print(f"No videos found in {VIDEO_FOLDER}")
        return
    for vf in video_files:
        in_path = os.path.join(VIDEO_FOLDER, vf)
        out_name = os.path.splitext(vf)[0] + '_inf.mp4'
        out_path = os.path.join(OUTPUT_FOLDER, out_name)
        process_video(model, in_path, out_path)

if __name__ == "__main__":
    main()


C:\Users\user\AppData\Local\Temp\ipykernel_3772\2381910914.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(checkpoint_path, map_location=device)

[testvideo1.mp4] Processed 50 frames...
[testvideo1.mp4] Processed 100 frames...
[testvideo1.mp4] Processed 150 frames...
[testvideo1.mp4] Processed 200 frames...
[testvideo1.mp4] Processed 250 frames...
[testvideo1.mp4] Processed 300 frames...
[testvideo1.mp4] Processed 350 frames...
[testvideo1.mp4] Processed 400 frames...
[testvideo1.mp4] Processed 450 frames...
[testvideo1.mp4] Processed 500 frames...
[testvideo1.mp4] Processed 550 frames...
[testvideo1.mp4] Processed 600 frames...
[testvideo1.mp4] Processed 650 frames...
Finished testvideo1.mp4, saved to video_results\testvideo1_inf.mp4
[testvideo2.mp4] Processed 50 frames...
[testvideo2.mp4] Processed 100 frames...
[testvideo2.mp4] Processed 150 frames...
[testvideo2.mp4] Processed 200 frames...
[testvideo2.mp4] Processed 250 frames...
[testvideo2.mp4] Processed 300 frames...
[testvideo2.mp4] Processed 350 frames...
[testvideo2.mp4] Processed 400 frames...
[testvideo2.mp4] Processed 450 frames...
[testvideo2.mp4] Processed 500 fr